In [5]:
from PIL import Image
import numpy as np
import torch
import imp
import sys
sys.path.append('/Projects/mk_utils/mklib/utils')
sys.path.append('/Projects/mk_utils/mklib/nn/')
from tcFaceFeatureExtract import tcFaceFeatureExtract
from pthnet.pthresnet import PthResNet

In [8]:
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks.tensorboard import *
from fastai.vision.gan import *
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageFile

In [2]:
torch.__version__

'1.0.1'

In [3]:
extractor_gpu=tcFaceFeatureExtract(model_net='train_7_58_pth_net_gpu.py',model_path='train_7_58_pth_model_gpu.pth', ctx='gpu')

In [6]:
myModel = PthResNet(3,112,512,[1,2,5,2],[64, 64, 128, 256, 512],res_ver='v3')

In [35]:
torch.save(myModel.cpu().state_dict(), '/Projects/test.pth')

In [15]:
print(myModel.cpu().state_dict())

OrderedDict()


In [16]:
model=torch.load('/Projects/test.pth')

In [18]:
model.keys()

odict_keys([])

In [10]:
y,z=myModel(x)

In [11]:
y.size()

torch.Size([10, 512])

In [12]:
z.size()

torch.Size([10, 512, 7, 7])

In [50]:
def recurse(func:Callable, x:Any, *args, **kwargs)->Any:
    if is_listy(x): return [recurse(func, o, *args, **kwargs) for o in x]
    if is_dict(x):  return {k: recurse(func, v, *args, **kwargs) for k,v in x.items()}
    return func(x, *args, **kwargs)

In [45]:
is_listy(list(myModel.children()))

True

In [49]:
list(list(myModel.children())[0][0].children())

[]

In [34]:
flt_model[1].weight.data.fill_(0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [35]:
list(myModel.children())[0][1].state_dict()['weight']

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
# the file should only keep layers with training parameters
def get_layer_name_for_weight_copy(file_name):
    layer_name = []
    with open(file_name,'r') as f:
        for line in f.readlines():
            line=line.strip().split('=')[0].strip()
            if len(line):
                layer_name += [line]
    #print(layer_name)
    return layer_name

In [10]:
# copy weights from .npy file to pytorch model, .npy file can be generated from mmdnn from mxnet model
# currently only support conv2d, batchnorm1d, batchnorm2d, prelu, linear
def copy_weights_to_pytorch_model(model, npy_file, layer_name_file):
    flt_model = fastai.torch_core.flatten_model(model)
    layer_names = get_layer_name_for_weight_copy(layer_name_file)    
    weights_dict = np.load(npy_file).item()
    layer_name_id = 0
    for layer in flt_model:
        print(layer_names[layer_name_id])
        if isinstance(layer,nn.Conv2d):
            name = layer_names[layer_name_id]
            layer.state_dict()['weight'].copy_(torch.from_numpy(weights_dict[name]['weights']))
            if 'bias' in weights_dict[name]:
                layer.state_dict()['bias'].copy_(torch.from_numpy(weights_dict[name]['bias']))
            layer_name_id += 1
        elif isinstance(layer,nn.BatchNorm1d) or isinstance(layer,nn.BatchNorm2d):
            name = layer_names[layer_name_id]
            if 'scale' in weights_dict[name]:                
                layer.state_dict()['weight'].copy_(torch.from_numpy(weights_dict[name]['scale']))
            else:
                layer.weight.data.fill_(1)
            if 'bias' in weights_dict[name]:
                layer.state_dict()['bias'].copy_(torch.from_numpy(weights_dict[name]['bias']))
            else:
                layer.bias.data.fill_(0)
            layer.state_dict()['running_mean'].copy_(torch.from_numpy(weights_dict[name]['mean']))
            layer.state_dict()['running_var'].copy_(torch.from_numpy(weights_dict[name]['var']))
            layer_name_id += 1
        elif isinstance(layer,nn.PReLU):
            name = layer_names[layer_name_id]
            layer.state_dict()['weight'].copy_(torch.from_numpy(weights_dict[name]['weights']))
            layer_name_id += 1
        elif isinstance(layer,nn.Linear):
            name = layer_names[layer_name_id]
            layer.state_dict()['weight'].copy_(torch.from_numpy(weights_dict[name]['weights']))
            if 'bias' in weights_dict[name]:
                layer.state_dict()['bias'].copy_(torch.from_numpy(weights_dict[name]['bias']))
            layer_name_id += 1
    assert layer_name_id == len(layer_names)
    return model

In [11]:
newModel = copy_weights_to_pytorch_model(myModel,'train_7_58_pth_wt_gpu.npy','FaceRecog_Resnet20_LayerName.txt')

stem_conv1
stem_bn2
stem_relu1
stage1_unit1_bn1
stage1_unit1_convr
stage1_unit1_bnr
stage1_unit1_conv1
stage1_unit1_bn2
stage1_unit1_prelu2
stage1_unit1_conv2
stage1_unit1_bn4
stage2_unit1_bn1
stage2_unit1_convr
stage2_unit1_bnr
stage2_unit1_conv1
stage2_unit1_bn2
stage2_unit1_prelu2
stage2_unit1_conv2
stage2_unit1_bn4
stage2_unit2_bn1
stage2_unit2_conv1
stage2_unit2_bn2
stage2_unit2_prelu2
stage2_unit2_conv2
stage2_unit2_bn4
stage3_unit1_bn1
stage3_unit1_convr
stage3_unit1_bnr
stage3_unit1_conv1
stage3_unit1_bn2
stage3_unit1_prelu2
stage3_unit1_conv2
stage3_unit1_bn4
stage3_unit2_bn1
stage3_unit2_conv1
stage3_unit2_bn2
stage3_unit2_prelu2
stage3_unit2_conv2
stage3_unit2_bn4
stage3_unit3_bn1
stage3_unit3_conv1
stage3_unit3_bn2
stage3_unit3_prelu2
stage3_unit3_conv2
stage3_unit3_bn4
stage3_unit4_bn1
stage3_unit4_conv1
stage3_unit4_bn2
stage3_unit4_prelu2
stage3_unit4_conv2
stage3_unit4_bn4
stage3_unit5_bn1
stage3_unit5_conv1
stage3_unit5_bn2
stage3_unit5_prelu2
stage3_unit5_conv2
stage3

In [12]:
torch.save(newModel.cpu(), 'Pytorch_NewModel.pth')

In [27]:
import numpy as np
x=torch.from_numpy(np.zeros((1,3,112,112)).astype(np.float32))

In [28]:
y1=extractor_gpu.model(x.cuda())

In [29]:
nModel = torch.load('Pytorch_NewModel.pth')

In [30]:
nModel = nModel.eval()

In [31]:
y,z = nModel(x)

In [32]:
np.max(np.abs(np.array(y)-np.array(y1.cpu())))

1.44355e-08

In [41]:
yp = np.array(y)[0]
yp1 = np.array(y1.cpu())[0]
yp = yp/np.linalg.norm(yp)
yp1 = yp1/np.linalg.norm(yp1)
np.dot(yp,yp1)

1.0